<a href="https://colab.research.google.com/github/prodbrown/SWIPE-TOOL/blob/main/RADIO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:

# ✅ Install gradio if needed
try:
    import gradio as gr
except ImportError:
    !pip install gradio
    import gradio as gr

import requests

# ✅ Base API
BASE_URL = "https://de1.api.radio-browser.info/json"

# ------------------------------------------
# ✅ 1. Get available countries
# ------------------------------------------
def get_countries():
    res = requests.get(f"{BASE_URL}/countries")
    res.raise_for_status()
    countries = sorted([c['name'] for c in res.json()])
    return countries

# ------------------------------------------
# ✅ 2. Get stations for selected country
# ------------------------------------------
def get_stations_in_country(country):
    res = requests.get(f"{BASE_URL}/stations/bycountry/{country}")
    res.raise_for_status()
    all_stations = res.json()

    # Filter stations with working mp3 links
    filtered = []
    for s in all_stations:
        name = s.get('name')
        url = s.get('url_resolved')
        if name and url and url.endswith(".mp3"):
            filtered.append((name, url))
    return filtered

# ------------------------------------------
# ✅ 3. Gradio function: Load stations
# ------------------------------------------
def load_stations(country):
    stations = get_stations_in_country(country)
    if not stations:
        return gr.update(choices=[]), "❌ No stations found in this country.", {}

    choices = [name for name, url in stations]
    urls = {name: url for name, url in stations}

    # ✅ FIX: Set default selection to first station
    return gr.update(choices=choices, value=choices[0]), f"✅ Found {len(choices)} stations in {country}.", urls

# ------------------------------------------
# ✅ 4. Gradio function: Play station
# ------------------------------------------
def play_station(station_name, station_dict):
    url = station_dict.get(station_name)
    if not url:
        return "❌ No stream found."

    return f"""
    <audio controls autoplay>
      <source src="{url}" type="audio/mpeg">
      Your browser does not support the audio element.
    </audio>
    """

# ------------------------------------------
# ✅ 5. Build Gradio Interface
# ------------------------------------------
with gr.Blocks() as app:
    gr.Markdown("# 🌍 **Radio Browser Player**\nListen to global radio stations by country")

    country_dropdown = gr.Dropdown(
        label="Choose Country",
        choices=get_countries(),
        interactive=True
    )

    load_btn = gr.Button("🔄 Load Stations")

    station_dropdown = gr.Dropdown(
        label="Stations",
        choices=[],
        interactive=True
    )
    info_output = gr.Textbox(label="Status", interactive=False)

    play_btn = gr.Button("▶️ Play")
    player_html = gr.HTML()

    # Hidden state to store station-name → URL mapping
    station_state = gr.State({})

    # Events
    load_btn.click(
        fn=load_stations,
        inputs=country_dropdown,
        outputs=[station_dropdown, info_output, station_state]
    )

    play_btn.click(
        fn=play_station,
        inputs=[station_dropdown, station_state],
        outputs=player_html
    )

# ✅ 6. Launch the app
app.launch()

It looks like you are running Gradio on a hosted a Jupyter notebook. For the Gradio app to work, sharing must be enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://d9735196f785372d6a.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
